## Neural Network
Extend `torch.nn.Module` to create a custom neural network.

Need to implement __init__ and forward methods.

To use the neural network, create an instance of the class and call it with input data.

In [34]:
import torch
class NeutralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            # first layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # second layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        return self.layers(x)

if __name__ == "__main__":
    model = NeutralNetwork(2, 1)
    x = torch.randn(1, 2)
    print(model(x))
    print(model)

tensor([[-0.3052]], grad_fn=<AddmmBackward0>)
NeutralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)


## Dataset and DataLoader
Extend `torch.utils.data.Dataset` to create a custom dataset.

Need to implement __init__, __len__, and __getitem__ methods.

DataLoader is used to create batches of data for training and testing.

In [35]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.targets[idx]
        return x, y

if __name__ == "__main__":
    # create custom datasets for training and testing
    # y = x[0] - x[1] > 0 ? 0 : 1
    X_train = torch.tensor([[-1.2, 3.1], [-0.9, 2.9], [-0.5, 2.6], [2.3, -1.1], [2.7, -1.5]])
    Y_train = torch.tensor([[0], [0], [0], [1], [1]])

    X_test = torch.tensor([[-0.8, 2.8], [2.6, -1.6], [1.0, 0.0], [0.0, 1.0]])
    Y_test = torch.tensor([[0], [1], [1], [0]])

    train_dataset = MyDataset(X_train, Y_train)
    test_dataset = MyDataset(X_test, Y_test)

    # use DataLoader to create batches for training and testing
    torch.manual_seed(42)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False, drop_last=True)

    # print batches
    for batch in train_loader:
        print(batch)
    for batch in test_loader:
        print(batch)

[tensor([[ 2.3000, -1.1000],
        [-0.5000,  2.6000]]), tensor([[1],
        [0]])]
[tensor([[-0.9000,  2.9000],
        [ 2.7000, -1.5000]]), tensor([[0],
        [1]])]
[tensor([[-0.8000,  2.8000],
        [ 2.6000, -1.6000],
        [ 1.0000,  0.0000],
        [ 0.0000,  1.0000]]), tensor([[0],
        [1],
        [1],
        [0]])]


## Training and Testing
`model.train()` means the model is in training mode.

`optimizer.zero_grad()` is used to clear the gradients of all optimized tensors.

`loss.backward()` is used to compute the gradients of the loss with respect to the model parameters.

`optimizer.step()` is used to update the model parameters based on the computed gradients.

`model.eval()` means the model is in evaluation mode.

In [38]:
import torch.nn.functional as F

torch.manual_seed(421)
model = NeutralNetwork(2, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.3)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for idx, (x, y) in enumerate(train_loader):
        logits = model(x)
        # loss = F.cross_entropy(logits, y)
        loss = F.binary_cross_entropy_with_logits(logits, y.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch+1}, Batch {idx+1}, Loss: {loss.item():.4f}")
    model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            logits = model(x)
            predictions = torch.round(torch.sigmoid(logits))
            accuracy = (predictions == y).float().mean()
            print(f"Test Accuracy: {accuracy.item():.4f}")


Epoch 1, Batch 1, Loss: 0.7159
Epoch 1, Batch 2, Loss: 0.6601
Test Accuracy: 0.5000
Epoch 2, Batch 1, Loss: 0.5264
Epoch 2, Batch 2, Loss: 0.2586
Test Accuracy: 0.5000
Epoch 3, Batch 1, Loss: 0.0932
Epoch 3, Batch 2, Loss: 0.4155
Test Accuracy: 0.7500
Epoch 4, Batch 1, Loss: 0.3278
Epoch 4, Batch 2, Loss: 0.1913
Test Accuracy: 1.0000
Epoch 5, Batch 1, Loss: 0.1021
Epoch 5, Batch 2, Loss: 0.0877
Test Accuracy: 1.0000


### Accuracy computation

In [39]:
def compute_accuracy(model, data_loader):
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for x, y in data_loader:
            logits = model(x)
            predictions = torch.round(torch.sigmoid(logits))
            total_correct += (predictions == y).sum().item()
            total_samples += y.size(0)
        accuracy = total_correct / total_samples
        return accuracy

print(f"Test Accuracy: {compute_accuracy(model, test_loader):.4f}")

Test Accuracy: 1.0000


## Save and Load Model
Use torch.save to save the model state_dict.

Use torch.load to load the model state_dict.

In [42]:
torch.save(model.state_dict(), "./models/first_model.pth")

modelLoaded = NeutralNetwork(2, 1)
modelLoaded.load_state_dict(torch.load("./models/first_model.pth"))
print(f"Test Accuracy: {compute_accuracy(modelLoaded, test_loader):.4f}")

Test Accuracy: 1.0000


## Using GPU
`tensor.to("cuda")` means move the tensor to GPU.

In [43]:
print(torch.cuda.is_available())

False
